# Projeto de Previsão de Atrasos de Voos

Este projeto visa prever atrasos de voos com base em várias características relacionadas a voos, usando um RandomForestClassifier. O notebook abrange aquisição de dados, análise exploratória de dados (EDA), engenharia de atributos, treinamento e avaliação do modelo.

## EDA_VOO (Análise Exploratória de Dados - Voo)

Esta seção abrange as etapas tomadas para adquirir o conjunto de dados, inspecionar seu conteúdo e realizar o carregamento inicial dos dados e a seleção de colunas relevantes.

### 1. Aquisição de Dados

Primeiro, interagimos com o Kaggle para listar os conjuntos de dados disponíveis e, em seguida, baixamos os dados específicos de voos para 2024.

In [ ]:
print("Listando conjuntos de dados do Kaggle (primeiras entradas):")
!kaggle datasets list | head

print("\nBaixando o conjunto de dados flight-data-2024...")
!kaggle datasets download -d hrishitpatil/flight-data-2024

print("\nArquivos no diretório atual após o download:")
!ls

### 2. Extração de Dados

O arquivo baixado é um arquivo ZIP, então precisamos descompactá-lo para acessar os arquivos CSV.

In [ ]:
print("Descompactando o conjunto de dados baixado:")
!unzip flight-data-2024.zip

print("\nArquivos no diretório atual após a descompactação:")
!ls

### 3. Inspeção e Carregamento Inicial de Dados

Inspecionamos os nomes das colunas do arquivo CSV principal (`flight_data_2024.csv`) para identificar os atributos relevantes para nossa tarefa de previsão. Em seguida, carregamos o conjunto de dados em um DataFrame pandas, selecionando apenas as colunas necessárias.

In [ ]:
import pandas as pd

# Inspeciona todas as colunas disponíveis no conjunto de dados
df_cols = pd.read_csv("flight_data_2024.csv", nrows=0)
print("Todas as colunas disponíveis em flight_data_2024.csv:")
print(df_cols.columns.tolist())

# Define as colunas a serem usadas para o modelo
colunas_usadas = [
    "op_unique_carrier",
    "origin",
    "origin_state_nm",
    "dest",
    "dest_state_nm",
    "distance",
    "month",
    "day_of_week",
    "crs_dep_time",
    "dep_delay"
]

# Carrega o conjunto de dados apenas com as colunas especificadas
df = pd.read_csv(
    "flight_data_2024.csv",
    usecols=colunas_usadas
)

print(f"\nFormato do conjunto de dados após o carregamento: {df.shape}")
print("Primeiras 5 linhas do DataFrame carregado:")
display(df.head())

## Feature_Engineering (Engenharia de Atributos)

Nesta seção, criamos a variável alvo e pré-processamos os atributos categóricos usando Label Encoding.

### 1. Criação da Variável Alvo

Uma nova coluna `atrasado` é criada, que é 1 se `dep_delay` (atraso na partida) for maior que 15 minutos, e 0 caso contrário.

In [ ]:
df["atrasado"] = (df["dep_delay"] > 15).astype(int)

print("Atraso na partida e variável alvo 'atrasado' para as primeiras 10 linhas:")
display(df[["dep_delay", "atrasado"]].head(10))

### 2. Codificação de Atributos Categóricos

As colunas categóricas são convertidas em representações numéricas usando `LabelEncoder`. Os mapeamentos são armazenados para uso posterior.

In [ ]:
from sklearn.preprocessing import LabelEncoder

colunas_categoricas = [
    "op_unique_carrier",
    "origin",
    "origin_state_nm",
    "dest",
    "dest_state_nm"
]

encoders = {}

for col in colunas_categoricas:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str)) # Garante que a coluna é do tipo string antes da codificação
    encoders[col] = le

print("Cabeçalho do DataFrame após a Codificação de Rótulos para atributos categóricos:")
display(df.head())

print("\nPrimeiras 5 linhas das colunas categóricas codificadas:")
display(df[colunas_categoricas].head())

## Treinamento_modelo

Esta seção detalha o processo de preparação dos dados para o modelo, treinamento de um RandomForestClassifier, avaliação de seu desempenho e salvamento do modelo treinado e dos mapeamentos de atributos.

### 1. Divisão dos Dados

O conjunto de dados é dividido em atributos (X) e a variável alvo (y). Em seguida, é dividido em conjuntos de treinamento e teste para avaliar o desempenho do modelo em dados não vistos.

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["dep_delay", "atrasado"])
y = df["atrasado"]

print("Atributos usados no modelo:")
print(X.columns.tolist())

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y # Estratifica para manter a distribuição de classes nas divisões
)

print(f"\nFormato dos atributos de treinamento (X_train): {X_train.shape}")
print(f"Formato dos atributos de teste (X_test): {X_test.shape}")
print(f"Formato da variável alvo de treinamento (y_train): {y_train.shape}")
print(f"Formato da variável alvo de teste (y_test): {y_test.shape}")

### 2. Definição e Treinamento do Modelo

Um RandomForestClassifier é inicializado com parâmetros específicos e então treinado usando os dados de treinamento.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

modelo = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1 # Usa todos os núcleos disponíveis
)

print("RandomForestClassifier inicializado. Treinando o modelo...")
modelo.fit(X_train, y_train)
print("Modelo treinado com sucesso.")

### 3. Avaliação do Modelo

O desempenho do modelo treinado é avaliado no conjunto de teste usando acurácia e um relatório de classificação.

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = modelo.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"Acurácia: {acc:.4f}")

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

### 4. Salvando o Modelo e Mapeamentos

O modelo treinado e os mapeamentos de atributos categóricos (LabelEncoders) são salvos em disco para uso futuro.

In [ ]:
import joblib
import json

# Salva o modelo treinado
joblib.dump(modelo, "modelo_atraso_voo_v1.joblib")
print("Modelo salvo como 'modelo_atraso_voo_v1.joblib'.")

# Prepara e salva os mapeamentos categóricos
mapeamento = {}
for col, le in encoders.items():
    mapeamento[col] = {
        classe: int(code)
        for classe, code in zip(le.classes_, le.transform(le.classes_))
    }

with open("mapeamento_categorias_v1.json", "w") as f:
    json.dump(mapeamento, f, indent=2)

print("Mapeamentos categóricos salvos como 'mapeamento_categorias_v1.json'.")

print("\nArquivos no diretório atual após salvar os artefatos:")
!ls